In [3]:
import xml.etree.ElementTree as ET
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from transformers import pipeline

In [ ]:
import nltk
nltk.download('words')
nltk.download("punkt")
nltk.download('maxent_ne_chunker')


In [42]:
import os
os.environ["NLTK_DATA"] = "C:/Users/Utilisateur/AppData/Roaming/nltk_data"


In [43]:
# Define file paths
input_file = 'sample-0001.xml'
output_file = 'sample.txt'
translation_output_file = 'sample_translated.txt'

In [44]:
# Step 1: Preprocessing the Dataset
def extract_abstract_texts(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    abstracts = []
    for abstract in root.findall(".//AbstractText"):
        abstracts.append(abstract.text)
    return abstracts

In [45]:

def segment_sentences(abstracts):
    sentences = []
    for abstract in abstracts:
        if abstract:
            sentences.extend(sent_tokenize(abstract))
    return sentences

In [46]:
abstracts = extract_abstract_texts(input_file)
sentences = segment_sentences(abstracts[:10])  # Adjust the number 10 based on your capacity

with open(output_file, 'w', encoding='utf-8') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [47]:
# Step 2: Part of Speech Tagging

def pos_tagging(sentences):
    pos_tags = [nltk.pos_tag(word_tokenize(sentence)) for sentence in sentences]
    return pos_tags

In [48]:

def compute_transition_matrices(tags):
    unigram_counts = Counter()
    bigram_counts = Counter()
    
    for tag_seq in tags:
        tag_seq = [tag for word, tag in tag_seq]  # Correcting the list comprehension to unpack tuples correctly
        unigram_counts.update(tag_seq)
        bigram_counts.update(nltk.bigrams(tag_seq))
    
    unigram_total = sum(unigram_counts.values())
    bigram_total = sum(bigram_counts.values())
    
    unigram_probs = {tag: count/unigram_total for tag, count in unigram_counts.items()}
    bigram_probs = {bigram: count/bigram_total for bigram, count in bigram_counts.items()}
    
    return unigram_probs, bigram_probs

In [49]:
pos_tags = pos_tagging(sentences)
pos_unigram_probs, pos_bigram_probs = compute_transition_matrices(pos_tags)

print("### Part of Speech Tagging Unigram Probabilities:")
print(pos_unigram_probs)


### Part of Speech Tagging Unigram Probabilities:
{'DT': 0.0889894419306184, 'JJ': 0.11161387631975868, 'NN': 0.16515837104072398, 'NNP': 0.05505279034690799, 'IN': 0.12518853695324283, 'NNS': 0.07993966817496229, 'VBD': 0.033182503770739065, '(': 0.014328808446455505, ')': 0.014328808446455505, 'CD': 0.04524886877828054, 'TO': 0.016591251885369532, 'VB': 0.016591251885369532, 'CC': 0.029411764705882353, ',': 0.03695324283559578, '.': 0.03770739064856712, 'VBN': 0.033936651583710405, 'NNPS': 0.0015082956259426848, 'VBP': 0.00980392156862745, 'RB': 0.02262443438914027, 'VBG': 0.008295625942684766, 'JJR': 0.004524886877828055, 'VBZ': 0.017345399698340876, '``': 0.0007541478129713424, "''": 0.0007541478129713424, 'PRP': 0.00904977375565611, 'WP': 0.0015082956259426848, 'RBR': 0.0007541478129713424, '$': 0.0007541478129713424, 'MD': 0.0022624434389140274, 'WDT': 0.005279034690799397, 'WRB': 0.0015082956259426848, 'EX': 0.0022624434389140274, ':': 0.0030165912518853697, 'PRP$': 0.0022624434

In [50]:

print("\n### Part of Speech Tagging Bigram Probabilities:")
print(pos_bigram_probs)


### Part of Speech Tagging Bigram Probabilities:
{('DT', 'JJ'): 0.0219435736677116, ('JJ', 'NN'): 0.05172413793103448, ('NN', 'NN'): 0.019592476489028215, ('NN', 'NNP'): 0.009404388714733543, ('NNP', 'NNP'): 0.0109717868338558, ('NNP', 'IN'): 0.0054858934169279, ('IN', 'NNS'): 0.006269592476489028, ('NNS', 'IN'): 0.025078369905956112, ('IN', 'NN'): 0.024294670846394983, ('NN', 'VBD'): 0.006269592476489028, ('VBD', 'IN'): 0.003918495297805642, ('IN', 'DT'): 0.039968652037617555, ('DT', 'NNP'): 0.007053291536050157, ('IN', 'NNP'): 0.007836990595611285, ('NNP', '('): 0.003918495297805642, ('(', 'NNP'): 0.009404388714733543, ('NNP', ')'): 0.0054858934169279, (')', 'IN'): 0.001567398119122257, ('NNP', 'CD'): 0.0054858934169279, ('CD', 'TO'): 0.0023510971786833857, ('TO', 'VB'): 0.009404388714733543, ('VB', 'DT'): 0.006269592476489028, ('DT', 'NNS'): 0.0109717868338558, ('NN', 'CC'): 0.0109717868338558, ('CC', 'JJ'): 0.006269592476489028, ('JJ', 'JJ'): 0.014890282131661442, ('NN', 'IN'): 0.

In [51]:
# Step 3: Named Entity Recognition

def ner_tagging(sentences):
    ner_tags = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        tagged_words = nltk.pos_tag(words)
        chunks = nltk.ne_chunk(tagged_words)
        for chunk in chunks:
            if hasattr(chunk, 'label'):
                ner_tags.append(chunk.label())
            else:
                ner_tags.append(chunk[1])
    return ner_tags

In [52]:

def compute_ner_transition_matrices(ner_tags):
    unigram_counts = Counter()
    bigram_counts = Counter()
    
    unigram_counts.update(ner_tags)
    bigram_counts.update(nltk.bigrams(ner_tags))
    
    unigram_total = sum(unigram_counts.values())
    bigram_total = sum(bigram_counts.values())
    
    unigram_probs = {tag: count/unigram_total for tag, count in unigram_counts.items()}
    bigram_probs = {bigram: count/bigram_total for bigram, count in bigram_counts.items()}
    
    return unigram_probs, bigram_probs

In [53]:
ner_tags = ner_tagging(sentences)
ner_unigram_probs, ner_bigram_probs = compute_ner_transition_matrices(ner_tags)

print("\n### Named Entity Recognition Unigram Probabilities:")
print(ner_unigram_probs)
print("\n### Named Entity Recognition Bigram Probabilities:")
print(ner_bigram_probs)

LookupError: 
**********************************************************************
  Resource [93mwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('words')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/words[0m

  Searched in:
    - 'C:\\Users\\Utilisateur/nltk_data'
    - 'C:\\Users\\Utilisateur\\anaconda3\\nltk_data'
    - 'C:\\Users\\Utilisateur\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Utilisateur\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Utilisateur\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# Step 4: Topics Extraction Using TF-IDF for Each Abstract
def extract_topics_for_each_abstract(abstracts, num_topics=3, num_words=5):
    topics_per_abstract = []
    vectorizer = TfidfVectorizer(stop_words='english', max_features=100)

    for abstract in abstracts:
        if abstract:  # Check if the abstract is not None or empty
            tfidf_matrix = vectorizer.fit_transform([abstract])
            feature_names = vectorizer.get_feature_names_out()
            
            # Sum the TF-IDF scores for each term in the abstract
            tfidf_scores = tfidf_matrix.sum(axis=0).A1
            tfidf_scores = dict(zip(feature_names, tfidf_scores))
            
            # Sort terms by their TF-IDF scores in descending order
            sorted_terms = sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True)
            
            # Extract the top terms as topics for the current abstract
            topics = [term for term, score in sorted_terms[:num_topics]]
            topics_per_abstract.append(topics)
        else:
            topics_per_abstract.append([])  # Handle empty abstracts

    return topics_per_abstract

In [ ]:

topics_per_abstract = extract_topics_for_each_abstract(abstracts[:10])  # Adjust the number 10 based on your capacity

In [ ]:

print("\n### Topics per Abstract:")
for i, topics in enumerate(topics_per_abstract, 1):
    print(f"Abstract {i}: {topics}")

In [ ]:

# Step 5: Translation
def translate_sentences(sentences, target_language='fr'):
    if target_language == 'fr':
        translator = pipeline('translation_en_to_fr', model='Helsinki-NLP/opus-mt-en-fr')
    elif target_language == 'ar':
        translator = pipeline('translation_en_to_ar', model='Helsinki-NLP/opus-mt-en-ar')
    else:
        raise ValueError("Unsupported target language. Use 'fr' for French or 'ar' for Arabic.")
    
    translations = translator(sentences)
    translated_sentences = [t['translation_text'] for t in translations]
    return translated_sentences

In [ ]:

# Example: Translate to French
translated_sentences = translate_sentences(sentences, target_language='fr')

In [ ]:

with open(translation_output_file, 'w', encoding='utf-8') as f:
    for sentence in translated_sentences:
        f.write(sentence + '\n')

In [ ]:


print("\n### Translated Sentences:")
for i, sentence in enumerate(translated_sentences, 1):
    print(f"Sentence {i}: {sentence}")


